In [ ]:
# ==============================================================================
# ETAPA 1: Entendimento do Problema e dos Dados
# ==============================================================================
print("--- INICIANDO ETAPA 1: Entendimento do Problema e dos Dados ---")

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Definir estilo dos gráficos
sns.set_theme(style="whitegrid")

# Carregar o conjunto de dados a partir do arquivo CSV
try:
    df = pd.read_csv('Base_M43_Pratique_LOL_RANKED_WIN.csv')
    print("Arquivo CSV carregado com sucesso!\n")
except FileNotFoundError:
    print("Erro: O arquivo 'Base_M43_Pratique_LOL_RANKED_WIN.csv' não foi encontrado.")
    exit() # Encerra o script se o arquivo não for encontrado

# Exploração Inicial dos Dados
print("--- Análise Inicial do DataFrame ---")
print("\n1. Visualização das 5 primeiras linhas:")
print(df.head())

print("\n2. Informações Gerais do DataFrame (tipos de dados e valores nulos):")
df.info()

print("\n3. Resumo Estatístico das Variáveis Numéricas:")
print(df.describe().T)

print("\n--- FIM DA ETAPA 1 ---")


# ==============================================================================
# ETAPA 2: Preparação dos Dados
# ==============================================================================
print("\n--- INICIANDO ETAPA 2: Preparação dos Dados ---")

# 1. Tratamento de Valores Faltantes (Verificação)
print("1. Verificação de valores nulos:")
print(df.isnull().sum())
print("Nenhum valor nulo encontrado. Nenhuma ação necessária.\n")

# 2. Remover 'gameId'
df = df.drop('gameId', axis=1)
print("2. Coluna 'gameId' removida do conjunto de dados.\n")

# 3. Separação das variáveis: Features (X) e Alvo (y)
X = df.drop('blueWins', axis=1)
y = df['blueWins']
print(f"3. Variáveis definidas: {X.shape[1]} features para prever a variável alvo 'blueWins'.\n")

# 4. Padronização dos Dados
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
print("4. Features padronizadas usando StandardScaler.")

print("\n--- FIM DA ETAPA 2 ---")


# ==============================================================================
# ETAPA 3: Análise Exploratória de Dados (EDA)
# ==============================================================================
print("\n--- INICIANDO ETAPA 3: Análise Exploratória de Dados ---")

# Análise de Correlação
correlation_matrix = df.corr()
blue_wins_correlation = correlation_matrix['blueWins'].sort_values(ascending=False)

print("1. Correlação das variáveis com a vitória do time azul ('blueWins'):")
print(blue_wins_correlation.head(10)) # Mostra as 10 mais positivas

# Remover a própria variável 'blueWins' da lista (correlação 1.0)
corr_to_plot = blue_wins_correlation.drop('blueWins')

# Pegar as 7 mais positivas e as 7 mais negativas
top_positive = corr_to_plot.head(7)
top_negative = corr_to_plot.tail(7)
top_correlations = pd.concat([top_positive, top_negative])

# Plotar o gráfico de barras
plt.figure(figsize=(12, 10))
sns.barplot(
    x=top_correlations.values,
    y=top_correlations.index,
    palette='coolwarm'
)
plt.title('Top Variáveis Correlacionadas com a Vitória (blueWins)', fontsize=16, pad=20)
plt.xlabel('Coeficiente de Correlação Pearson')
plt.ylabel('Variável')
plt.tight_layout()
plt.show()

print("\n2. Gráfico de correlação (Top Positivas e Negativas) gerado.")
print("\nInsights: 'blueGoldDiff' e 'blueExperienceDiff' têm a correlação mais forte com a vitória.")

print("\n--- FIM DA ETAPA 3 ---")


# ==============================================================================
# ETAPA 4: Seleção de Modelos e Algoritmos
# ==============================================================================
print("\n--- INICIANDO ETAPA 4: Seleção de Modelos e Algoritmos ---")

from sklearn.model_selection import train_test_split

# 1. Divisão dos dados em Treinamento e Teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
print(f"1. Dados divididos em conjuntos de treino e teste (80/20).\n")

# 2. Escolha de Algoritmos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

models = {
    "Regressão Logística": LogisticRegression(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}
print("2. Modelos selecionados: Regressão Logística, Random Forest e Gradient Boosting.")

print("\n--- FIM DA ETAPA 4 ---")


# ==============================================================================
# ETAPA 5: Treinamento e Avaliação do Modelo
# ==============================================================================
print("\n--- INICIANDO ETAPA 5: Treinamento e Avaliação do Modelo ---")

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report

results = {}

# Treinamento e avaliação iterativa
for name, model in models.items():
    print(f"\n--- Treinando e Avaliando: {name} ---")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    # Armazenar métricas
    results[name] = {
        "Acurácia": accuracy_score(y_test, y_pred),
        "Precisão": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred),
        "AUC-ROC": roc_auc_score(y_test, y_proba)
    }
    print(classification_report(y_test, y_pred))

# Comparação dos Modelos
print("\n--- Comparação Final do Desempenho dos Modelos ---")
results_df = pd.DataFrame(results).T
print(results_df.sort_values(by="Acurácia", ascending=False))

print("\n--- FIM DA ETAPA 5 ---")


# ==============================================================================
# ETAPA 6 e 7: Ajuste Final e Interpretação dos Resultados
# ==============================================================================
print("\n--- INICIANDO ETAPA 6 e 7: Ajuste Final e Interpretação ---")

# Seleciona o melhor modelo treinado (Random Forest)
best_model = models["Random Forest"]

# Interpretação dos Resultados - Importância das Features
feature_importances = pd.DataFrame({
    'feature': X.columns,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n1. Importância das Features (de acordo com o Random Forest):")
print(feature_importances.head(10)) # Mostrando as 10 mais importantes

# --- GRÁFICO 2  ---
# Gráfico de barras horizontais

top_10_features = feature_importances.head(10)

plt.figure(figsize=(12, 8))
sns.barplot(
    x='importance',
    y='feature',
    data=top_10_features,
    palette='mako'
)
plt.title('As 10 Variáveis Mais Importantes para o Modelo (Random Forest)', fontsize=16, pad=20)
plt.xlabel('Nível de Importância')
plt.ylabel('Variável')
plt.tight_layout()
plt.show()

print("\n2. Gráfico com as 10 features mais importantes gerado.")

print("\n--- Relatório Final e Recomendações ---")
print("""
Análise e Resultados do Projeto:

Esta é a conclusão mais importante para o Tech Lead da Riot Games, vinda da "Etapa 6 e 7".

 O modelo de Random Forest nos diz que as variáveis mais importantes para
prever a vitória são, de longe, blueGoldDiff (diferença de ouro) e blueExperienceDiff
(diferença de experiência).

 Redundância Confirmatória: Note que redGoldDiff e redExperienceDiff também estão no topo.
Elas são apenas o "espelho" das variáveis do time azul (blueGoldDiff é o oposto exato de
redGoldDiff). O fato de ambas aparecerem no topo apenas reforça que a diferença de recursos
entre os times é o fator mais crítico.

 O "Farm" é Decisivo: As 10 variáveis mais importantes estão todas relacionadas a Ouro (Gold)
 e Experiência (Experience). Isso significa que, nos primeiros 10 minutos, a vantagem econômica
 (obtida por "farmar" minions e conseguir abates) é um indicador de vitória muito mais forte
 do que outros objetivos (como o primeiro dragão ou arauto, que não aparecem nesse top 10).

 Resumo da Conclusão Final (Para o Stakeholder):
"Conseguimos criar um modelo que prevê corretamente o vencedor de 7 em cada 10 partidas usando
apenas os dados dos primeiros 10 minutos. A análise confirma que a vantagem econômica é o fator
decisivo no início do jogo; a equipe que estabelece uma liderança em ouro e experiência tem
uma probabilidade de vitória drasticamente maior."
""")

print("\n--- PROJETO CONCLUÍDO ---")